In [1]:
import http.client
import json
import pandas as pd
import openpyxl



def tratar_dados_aninhados(dados):
    if 'atividade_principal' in dados:
        dados['atividade_principal'] = ';'.join([ativ['text'] for ativ in dados['atividade_principal']])
    return dados                                            
                                              


def obter_dados_empresa_por_cnpj(cnpj):

    conexao = http.client.HTTPSConnection('www.receitaws.com.br')
    
    conexao.request('GET', f'/v1/cnpj/{cnpj}')
    
    resposta = conexao.getresponse()
    
    if resposta.status != 200:
        
        return{'Status': 'ERROR', 'Message': f'Resposta HTTP com status {resposta.status} '}
    else:
        dados = resposta.read()
    
        empresa = json.loads(dados.decode('utf-8'))
    
        conexao.close()
        
        try:
          
            empresa = json.loads(dados.decode('utf-8'))
            
            print(f'Empresa decodificada: {empresa}')
            
            return empresa
          
          
        except json.JSONDecodeError as e:
          
            print(f'Erro na decodificação do JSON: {str(e)}')
            
            return {'status': 'ERROR', 'Message': 'Erro na decodificação do JSON'}


caminho_planilha = 'dados_empresa.xlsx' 

planilha_cnpjs = pd.read_excel(caminho_planilha, sheet_name='cnpjs')

cnpjs = planilha_cnpjs['CNPJ'].dropna()

resultados = pd.DataFrame(columns=['cnpj', 'nome', 'telefone', 'email', 'logradouro', 'bairro', 'municipio', 'uf', 'cep', 'atividade principal' ])

for cnpj in cnpjs:
  
  endereco = obter_dados_empresa_por_cnpj(str(cnpj).replace('-', ''))
  
  if endereco:

      atividade_principal = '; '.join([ativ.get('text', '') for ativ in endereco.get('atividade_principal', [])])
    
      nova_linha = pd.DataFrame([{
          'cnpj': cnpj,
          'nome':
            endereco.get('nome', ''),
          'telefone': endereco.get('telefone', ''),
          'email': endereco.get('email', ''),
          'logradouro': endereco.get('logradouro', ''),
          'bairro': endereco.get('bairro', ''),
          'municipio': endereco.get('municipio', ''),
          'uf': endereco.get('uf', ''),
          'cep': endereco.get('cep',''), 
          'atividade principal': atividade_principal
          
      }])
      
      resultados = pd.concat([resultados, nova_linha], ignore_index=True)
      
with pd.ExcelWriter(caminho_planilha, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
  
    resultados.to_excel(writer, sheet_name='Resultados', index=False)
    
print('Endereços salvos na aba Dados')

Empresa decodificada: {'abertura': '22/07/2009', 'situacao': 'ATIVA', 'tipo': 'MATRIZ', 'nome': 'FUNDO DE GARANTIA DE OPERACOES', 'fantasia': 'FUNDO DE GARANTIA DE OPERACOES', 'porte': 'DEMAIS', 'natureza_juridica': '324-7 - Fundo Privado', 'atividade_principal': [{'code': '64.99-9-99', 'text': 'Outras atividades de serviços financeiros não especificadas anteriormente'}], 'logradouro': 'SETOR SAUN QUADRA 5 LOTE B TORRE I', 'numero': 'SN', 'complemento': 'ANDAR: 10;', 'municipio': 'BRASILIA', 'bairro': 'ASA NORTE', 'uf': 'DF', 'cep': '70.040-912', 'email': 'governo@bb.com.br', 'telefone': '(61) 3493-0300 / (61) 3310-8813', 'data_situacao': '22/07/2009', 'cnpj': '10.983.890/0001-52', 'ultima_atualizacao': '2024-12-14T23:59:59.000Z', 'status': 'OK', 'efr': '', 'motivo_situacao': '', 'situacao_especial': '', 'data_situacao_especial': '', 'atividades_secundarias': [{'code': '00.00-0-00', 'text': 'Não informada'}], 'capital_social': '0.00', 'qsa': [], 'simples': {'optante': False, 'data_opca